In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
id        20800 non-null int64
title     20242 non-null object
author    18843 non-null object
text      20761 non-null object
label     20800 non-null int64
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
df=df.dropna()

In [6]:
X=df.drop('label',axis=1)
Y=df.label

In [7]:
print(X.shape)
print(Y.shape)

(18285, 4)
(18285,)


In [54]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding,LSTM, Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot


Onehot Representation

In [10]:
sentences=X.copy()
sentences.reset_index(inplace=True)

TExt processing

In [12]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [16]:
ps=PorterStemmer()
corpus=[]
for i in range(len(sentences)):
    review=re.sub('^a-zA-Z',' ',sentences['title'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [19]:
corpus[4]


'iranian woman jail fiction unpublish stori woman stone death adulteri'

In [20]:
voc_size=5000

In [21]:
onehot_rep=[one_hot(words,voc_size) for words in corpus]

In [40]:
#find the maximam length
l=[]
for i in range(len(onehot_rep)):
    lenth=len(onehot_rep[i])
    l.append(lenth)
    
import numpy as np
lenth_array=np.array(l)
sent_length=lenth_array.max()
sent_length

47

Padding

In [41]:
embedded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 2107, 4416, 3031],
       [   0,    0,    0, ..., 2656,  494, 2037],
       [   0,    0,    0, ..., 3364, 3089, 1199],
       ...,
       [   0,    0,    0, ..., 3982, 3317, 4537],
       [   0,    0,    0, ..., 2321, 3018, 3533],
       [   0,    0,    0, ...,  909, 4825,  700]])

Create model

In [55]:
embedding_vector_feature=50

model=Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [56]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 47, 50)            250000    
_________________________________________________________________
dropout (Dropout)            (None, 47, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________


In [57]:
print(len(embedded_docs))
print(Y.shape)

18285
(18285,)


In [58]:
print(type(embedded_docs))
print(type(Y))

<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


In [59]:
import numpy as np
x_final=np.array(embedded_docs)
y_final=np.array(Y)

In [60]:
x_final.shape,y_final.shape

((18285, 47), (18285,))

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

Train model

In [62]:
model.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 11s 24ms/step - loss: 0.4107 - accuracy: 0.7798 - val_loss: 0.1985 - val_accuracy: 0.9183
Epoch 2/10
383/383 [==============================] - 8s 22ms/step - loss: 0.1244 - accuracy: 0.9533 - val_loss: 0.2110 - val_accuracy: 0.9176
Epoch 3/10
383/383 [==============================] - 9s 24ms/step - loss: 0.0648 - accuracy: 0.9783 - val_loss: 0.2337 - val_accuracy: 0.9117
Epoch 4/10
383/383 [==============================] - 11s 28ms/step - loss: 0.0381 - accuracy: 0.9893 - val_loss: 0.3188 - val_accuracy: 0.9123
Epoch 5/10
383/383 [==============================] - 10s 27ms/step - loss: 0.0240 - accuracy: 0.9922 - val_loss: 0.3432 - val_accuracy: 0.9047
Epoch 6/10
383/383 [==============================] - 11s 27ms/step - loss: 0.0153 - accuracy: 0.9964 - val_loss: 0.4260 - val_accuracy: 0.9107
Epoch 7/10
383/383 [==============================] - 9s 24ms/step - loss: 0.0064 - accuracy: 0.9987 - val_loss: 0.4997 - val_accuracy: 0.


Performance Metrics And Accuracy

In [65]:
pred=model.predict_classes(X_test)

C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [67]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [68]:
confusion_matrix(y_test,pred)

array([[3083,  336],
       [ 223, 2393]], dtype=int64)

In [71]:
accuracy_score(y_test,pred)*100

90.73736536868269